# 以命令作業身分執行定型腳本

您可以使用適用于 Azure Machine Learning 的 Python SDK，將腳本提交為命令作業。 藉由使用作業，您可以輕鬆地追蹤定型機器學習模型時的輸入參數和輸出。

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格，以確認其已安裝。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 加以安裝。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，您現在可以連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 起始命令作業

執行下列資料格來定型分類模型以預測糖尿病。 模型會執行可在**src**資料夾中找到**的 train-model-parameters.py**腳本來定型。 它會使用 **diabetes.csv** 檔案作為定型資料。 

- `code`：指定要執行之腳本的資料夾。
- `command`：指定要完全執行的內容。
- `environment`：指定在執行 命令之前，要在計算上安裝的必要套件。
- `compute`：指定要用來執行命令的計算。
- `display_name`：個別作業的名稱。
- `experiment_name`：作業所屬的實驗名稱。

請注意，用來測試終端機中腳本的命令與下列作業組態中的命令相同。 

In [ ]:
# enter details of your AML workspace
subscription_id = "YOUR-SUBSCRIPTION-ID"
resource_group = "rg-dp100-labs"
workspace = "mlw-dp100-labs"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)